---

title: Linux PWM 的使用

keywords: maixpy3, PWM, Linux

desc: maixpy3 doc: PWM 的使用

---

| 更新时间 | 负责人 | 内容 | 备注 |
| --- | --- | --- | --- |
| 2022年1月4日 | Rui | 初次编写文档 | --- |
| 2022年1月8日 | Rui | 修改文档的编写方式 | 使用 Jupyter notebook 进行编写文档 |
| 2022年1月18日 | Rui | 修改文档，增加效果图 | 外设文档通过测试的平台有 MaixII-Dock,使用的是 MaixPy3 0.4.0 |


MaixPy3 把 Linux 系统中的 PWM 的使用方法进行封装和简化使用，让用户使用起来更加的简单

## 使用方式

PWM 的使用方式和 GPIO 的类似，需要根据管脚定义图确定所使用的 PWM 通道序号，下面以 MaixII-Dock 为例子讲述如何使用 MaixPy3 中的 PWM 

### 准备

查看开发板的管脚定义图，选择 PWM 输出通道序号，PWM-x 是指 PWM 的输出通道序号

<img src="./../asserts/M2Dock_pin.jpg" width=450px>
<img src="./../asserts/R329-pin.jpg" height=450px>

这里选择使用 PWM-6

### 开始

先将 PWM-6 实例化，设置周期和占空比，最后是使能 PWM，就可以输出了，具体代码看下面

In [1]:
from maix import pwm
import time
pwm6 = pwm.PWM(6)
pwm6.export()
pwm6.period = 20000000  # 表示 pwm 的周期，单位 ns
pwm6.duty_cycle = 500000  # 表示占空比，单位 ns
pwm6.enable = True        # 表示是否使能 pwm
for t in range (3):
    for i in range(500000, 15000000, +100000):
        pwm6.duty_cycle = i
        time.sleep(0.05)
    for i in range(15000000, 500000, -100000):
        pwm6.duty_cycle = i
        time.sleep(0.05)

[ rpyc-kernel ]( running at Mon Jan 17 16:58:42 2022 )



Traceback (most recent call last):
  File "<string>", line unknown, in <module>
Remote.KeyboardInterrupt




别的开发板使用方式是同样的流程，包括树莓派

> 注明：
> 建议小白使用在对应开发管脚定义图上的对应 PWM 通道，别的通道需要自行查看 [Linux PWM](https://www.baidu.com/s?ie=UTF-8&wd=Linux%20PWM) 使用方法。


## PWM 用途

1. 呼吸灯
    
    ![PWM](./../asserts/pwm.gif)
2. PWM 可以作为电机驱动控制信号，驱动舵机。但是外接电机的时候需要做好电源隔离，不要直接将舵机接到开发板上，舵机产生的反向电流会把开发板上的芯片给击穿。

    ![](./../asserts/PWM_1.gif)

## 了解更多

更多关于 Linux PWM 的可以查看大老鼠写的[博客](https://www.cnblogs.com/juwan/p/14343977.html)